In [ ]:
import os
from os.path import join
import requests
import json
import urllib
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from slugify import slugify
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from pytube import YouTube

In [ ]:
song = """Ella existio solo en un sueño 
el es un poema que el poeta 
nunca escribió 
y en la eternidad los dos 
unieron sus almas para darle vida 
"""

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
spanish_stopwords = set(list(punctuation)) 

In [ ]:
# Cleaning song
song_wo_stopwords = [word.lower() for word in word_tokenize(song) if word.lower() not in spanish_stopwords]
print(song_wo_stopwords)

In [ ]:
# Load words:
candidates = {}

for w in song_wo_stopwords:
    candidates[w] = []
    
directory = 'youtube-captions/captions'
for file in os.listdir(directory):
    if file.endswith("json"):
        with open(join(directory, file), 'r') as captions_file:
            video = json.load(captions_file)
            captions = video['captions_parsed']
            for caption in captions:
                if caption['content'] == None:
                    continue
                tokenized = word_tokenize(caption['content'])
                for w1 in tokenized:
                    if w1.lower() in candidates:
                        caption['id'] = video['id']
                        caption['count'] = len(tokenized)
                        candidates[w1.lower()].append(caption)

In [ ]:
complete_df = pd.read_csv("youtube-captions/complete.csv", index_col=0,parse_dates=['published_at'])
print(complete_df.info())
complete_df.head()

In [ ]:
def start_to_time(s: float):
    seconds = float(s) / 60
    ceil_seconds = math.floor(seconds)
    minutes = ceil_seconds
    seconds = round((seconds - ceil_seconds) * 60)
    return str(minutes) +"m" + str(seconds)+ "s"

In [ ]:
import math

video_url = "http://youtube.com/watch?v=%s&t=%s"
for candidate_key in candidates:
    print(candidate_key)
    if len(candidates[candidate_key]) == 0:
        continue
    candidates[candidate_key].sort(key=lambda x: x['count'], reverse=True)
    for c in candidates[candidate_key][:4]:
        tokenized = word_tokenize(c['content'])
        word_count = len(tokenized)
        duration, start = float(c['duration']), float(c['start'])
        word_duration = word_count / duration
        word_location = tokenized.index(candidate_key)
        tentative_word_start = (start + word_location * word_duration) - 1
        print(video_url % (c['id'], start_to_time(tentative_word_start)))
#        print(word_duration)
#        print(video_url % (c['id'], start_to_time(text['start'])))